In [1]:
import numpy as np
from itertools import combinations
from numpy import random
import math

In [21]:
# return a list of all size k subset of [n]
def get_coord(n, k):
    temp = [i +1 for i in range(n)]
    return list(combinations(temp, k))

# generate cov matrix 
# n is number of vertices 
# vals[i] is the covariance of S, S' when intersection is of size i (e.g. we should have vals 0, 1, 2, 3)
def cov_mat(n, k, vals):
    coordinate = get_coord(n, k)
    l_c = len(coordinate)
    cov_mat = np.zeros((l_c, l_c))
    for i in range(l_c):
        cov_mat[i][i] = vals[k]
        for j in range(i):
            size_inter = len(set(coordinate[i]).intersection(coordinate[j]))
            cov_mat[i][j] = vals[size_inter]
            cov_mat[j][i] = vals[size_inter]
    return cov_mat
    
def check_psd(M):
    try:
        np.linalg.cholesky(M)
        return True
    except np.linalg.LinAlgError:
            return False

def sample_gaussain(mean, M):
    return random.multivariate_normal(mean, M)

def round(x):
    return [c >= 1/2 for c in x]
# get adj matrices from sampled triangles
# x is the sample, V is the index of subsets, n is the number of vertices 
# x and V should have length (n choose 3)
def get_graph(x, V, n):
    res = np.zeros((n, n))
    for i in range(len(V)):
        if x[i] == 1:
            cur = V[i]
            for (p, q) in list(combinations(cur, 2)):
                res[p - 1][q - 1] = 1
                res[q - 1][p - 1] = 1
    return res

def get_mean_var(vals, n):
#     means = vals[-1] * np.ones(n)
    # get covariance
    vals -= vals[-1] ** 2
    return vals

In [3]:
# test the covariance matrix put right values based on size of intersection
def test_cov(M, V, vals, i, j):
    print(V[i], V[j])
    size_int = len([a for a in V[i] if a in V[j]])
    print(M[i][j] == vals[size_int])

In [4]:
# set parameter

n = 10
k = 3
p = .5

# get list of all size k subsets
V = get_coord(n, k)

# writedown list of moments vals[i] is E[S,S'] when |S \cap S'| = i
vals = np.array([p**6, p**6, p**5, p**3])

means = vals[-1] * np.ones(len(V))
# get covariance
vals -= vals[-1] ** 2

cov = cov_mat(n, k, vals)
# check covariance is psd
check_psd(cov)

True

In [5]:
test_cov(cov, V, vals, 0, 1)

(1, 2, 3) (1, 2, 4)
True


In [6]:
samples = sample_gaussain(means, cov)

In [7]:
adj = get_graph(round(samples), V, n)

In [15]:
p = 3 
s = 3

m = p**s
N = int(3 * m **3 *(m**3 - 1) * (m ** 2 - 1) / p**7)


In [19]:
print(N)

58240


In [16]:
math.comb(N - 1, 3)

9650323135994755894524935

In [27]:
p = 3 
s = 3

m = p**s
N = int(3 * m **3 *(m**3 - 1) * (m ** 2 - 1) / p**7)
# print(N)

# V = get_coord(N, 3)

# prob of intersection size 3 -> triangle
p3 = 2 * p **7 /((N - 1)* (N - 2)) 
p2 = p ** 2 * (p ** 2 - 1) * p ** 5/ math.comb(N - 1, 3)
p1 = p ** 7 * (p ** 7 - 2 * p **2 + 1)/ (2 * math.comb(N - 1, 4))
p0 = p3 ** 2

vals = np.array([p0, p1, p2, p3])

print(vals)


[8.54083005e-28 2.54233216e-27 1.81299629e-21 2.92246986e-14]


In [28]:
variance = get_mean_var(vals, math.comb(N, 3))

In [29]:
print(variance)

[0.00000000e+00 1.68824915e-27 1.81299543e-21 2.92246986e-14]


In [ ]:
cov = cov_mat(N, 3, variance)